### WBS Coding School
___
### --PROJECT--
# Solar Up: Solar Panel Object Detection
## Model training for image segmentation of solar panels

Use Google's GPU to run this script to substantially reduce training time.
(Next to "Connection" -> Click arrow -> "Change runtime type" -> choose a GPU, e.g. "T4 GPU")
___

#### Libraries

In [ ]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 5.4 MB/s eta 0:00:00


In [2]:
import os
import shutil

import ultralytics
from ultralytics import YOLO
from ultralytics import settings

## 1.&nbsp; Download Data

In [3]:
dataset_name = "kasmi_solar"

Check hardware and directory structure. Make sure a GPU is being used for model training.

In [4]:
ultralytics.checks()

Ultralytics YOLOv8.1.1 🚀 Python-3.11.7 torch-2.1.0 CPU (Apple M2 Pro)
Setup complete ✅ (12 CPUs, 16.0 GB RAM, 739.0/926.4 GB disk)


In [10]:
# Define main directories
main_dirs = ["train", "test", "val"]

for main_dir in main_dirs:
    # Define the source directories
    img_dir = f"datasets/{dataset_name}/{main_dir}/img_with_panels"
    label_dir = f"datasets/{dataset_name}/{main_dir}/labels_segmentation/normalized"

    # Define the destination directories
    dest_img_dir = f"datasets/{dataset_name}/{main_dir}/images"
    dest_label_dir = f"datasets/{dataset_name}/{main_dir}/labels"

    # Create the destination directories if they don't exist
    os.makedirs(dest_img_dir, exist_ok=True)
    os.makedirs(dest_label_dir, exist_ok=True)

    # Copy all the files from the image and label directories to the destination directories
    for filename in os.listdir(img_dir):
        shutil.copy(os.path.join(img_dir, filename), dest_img_dir)

    for filename in os.listdir(label_dir):
        shutil.copy(os.path.join(label_dir, filename), dest_label_dir)

## 2. Model training

Here we'll load a pretrained YOLOv8 model (small, medium or large) and train it on the solar panel satellite data. Subsequently, we'll check its performance on the validation set.

In [16]:
# Load a COCO-pretrained YOLOv8m model
model_name = "yolov8n-seg"
model = YOLO(f"{model_name}.pt")

100%|██████████| 6.73M/6.73M [00:01<00:00, 4.52MB/s]


In [11]:
# Display model information (optional)
model.info()

YOLOv8l-seg summary: 401 layers, 45997728 parameters, 0 gradients, 221.1 GFLOPs


(401, 45997728, 0, 221.13428480000005)

In [17]:
# Train the model on our custom dataset
config_file = f"{dataset_name}_segmentation.yaml"
epochs = 50
img_size = 400
experiment = f"{model_name}_{epochs}"
device = "mps"  # 0 = cuda, i.e. the GPU device; "cpu" = cpu

results = model.train(
    data=config_file, epochs=epochs, imgsz=img_size, name=experiment, device=device
)

New https://pypi.org/project/ultralytics/8.1.2 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.1 🚀 Python-3.11.7 torch-2.1.0 MPS (Apple M2 Pro)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=kasmi_solar_segmentation.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=400, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=yolov8n-seg_50, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

train: Scanning /Users/sanjaboekle/Dropbox/12_Programmieren/0_WBS_data_science_bootcamp/10_final_project/solar_panel_object_detection/datasets/kasmi_solar/train/labels.cache... 27936 images, 18624 backgrounds, 0 corrupt: 100%|██████████| 46560/46560 [00:00<?, ?it/s]
val: Scanning /Users/sanjaboekle/Dropbox/12_Programmieren/0_WBS_data_science_bootcamp/10_final_project/solar_panel_object_detection/datasets/kasmi_solar/val/labels.cache... 5988 images, 3992 backgrounds, 0 corrupt: 100%|██████████| 9980/9980 [00:00<?, ?it/s]

Plotting labels to runs/segment/yolov8n-seg_50/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.777      3.358      4.321      1.613         23        416:   0%|          | 7/2910 [06:25<44:27:52, 55.14s/it]


KeyboardInterrupt: 

## 3. Evaluate Model

In [ ]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered

metrics.box.map  # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps  # a list contains map50-95(B) of each category
metrics.seg.map  # map50-95(M)
metrics.seg.map50  # map50(M)
metrics.seg.map75  # map75(M)
metrics.seg.maps  # a list contains map50-95(M) of each category

## 4. Export Model Weights

In [ ]:
# Load a model
path_to_best = f"runs/detect/{experiment}/weights/best.pt"  # .pt = PyTorch model
model = YOLO(path_to_best)  # load a custom trained model

# Export the model
model.export()  # the default format is 'torchscript' (PyTorch)

In [ ]:
# Specify the output path for the ZIP archive
output_path = f"models/trained_model_{experiment}"
print(output_path)

shutil.make_archive(output_path, "zip", f"/content/runs/detect/{experiment}")

In [ ]:
# Download model weights:
files.download(f"runs/detect/{experiment}/weights/best.torchscript")

In [ ]:
files.download(f"models/trained_model_{experiment}.zip")